In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render ul li{font-size:22pt; line-height:30px;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

In [2]:
# ===============================
# 📌 기본 라이브러리
# ===============================
import pandas as pd
import numpy as np

# ===============================
# 📌 시각화 라이브러리
# ===============================
import matplotlib.pyplot as plt
import seaborn as sns

# ===============================
# 📌 경고 메시지 제거 (대용량 CSV 처리 시 깔끔)
# ===============================
import warnings
warnings.filterwarnings("ignore")

# ===============================
# 📌 한글 폰트 설정 (Windows)
# ===============================
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# ===============================
# 📌 CSV에서 불러올 컬럼 지정
# (불필요한 컬럼 제거 → 메모리 절약)
# ===============================
use_cols = [
    '인허가일자',
    '상세영업상태명',
    '폐업일자',
    '소재지전체주소',
    '도로명전체주소',
    '업태구분명',
    '좌표정보x(epsg5174)',
    '좌표정보y(epsg5174)',
    '사업장명',
    '시설총규모'
    
]


In [4]:
# ===============================
# 📌 데이터 로드
# ===============================
df = pd.read_csv(
    r"C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점.csv",
    encoding="cp949",
    usecols=use_cols,
    low_memory=False
)

In [5]:
# ✅ 2000년 ~ 2022년까지만 사용 (금리 데이터가 2000년 2월부터라 2월부터 사용. 2022년 12월 데이터는 분류할대 3으로 처리 되기때문에.)
df = df[
    (df['인허가일자'] >= '2000-02-01') &
    (df['인허가일자'] <= '2022-11-31')
].copy()


In [6]:
#이상치 제거
(df['시설총규모'] <= 0).sum()

5768

In [7]:
# ============================================================
# [단락 1] 라이브러리
# ============================================================
import pandas as pd
import re


In [8]:
# ============================================================
# [단락 2] 데이터 로드
# ============================================================
in_path = r"C:\ai\lecNote\1st_Project\data\고용지표_20251224100329.csv"   # ✅ 너 파일명/경로로 수정
df = pd.read_csv(in_path, encoding="utf-8", low_memory=False)

print("원본 컬럼:", df.columns.tolist())
print("원본 상위 5개:\n", df.head())


원본 컬럼: ['시점', '경제활동참가율', '경제활동참가율.1', '경제활동참가율.2', '실업률', '실업률.1', '실업률.2', '고용률', '고용률.1', '고용률.2']
원본 상위 5개:
          시점 경제활동참가율 경제활동참가율.1 경제활동참가율.2  실업률 실업률.1 실업률.2   고용률 고용률.1 고용률.2
0        시점      합계        남자        여자   합계    남자    여자    합계    남자    여자
1  2000. 01    61.6      75.3      48.6  6.7   7.5   5.4  57.5  69.6  46.0
2  2000. 02    61.1      74.6      48.4  6.5   7.2   5.4  57.2  69.2  45.8
3  2000. 03    60.7      74.1      48.0  5.9   6.2   5.4  57.1  69.5  45.5
4  2000. 04    60.7      74.3      47.9  5.3   5.9   4.3  57.5  69.9  45.8


In [9]:
# ============================================================
# [단락 3] 사용할 컬럼 지정 (합계/전체로 보이는 기본 컬럼만)
#  - '경제활동참가율.1' 같은 중복 컬럼은 보통 성별/세부 구분이라 제외
# ============================================================
base_cols = ["경제활동참가율", "실업률", "고용률"]

missing = [c for c in base_cols if c not in df.columns]
if missing:
    raise ValueError(f"필수 컬럼이 없습니다: {missing}")


In [10]:
# ============================================================
# [단락 4] 숫자 변환 (문자/공백 섞여도 안전하게)
# ============================================================
for c in base_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")


In [11]:
# ============================================================
# [단락 5] 헤더가 데이터로 들어간 1줄 제거
#  - '시점' 컬럼에 4자리 연도(예: 2000)가 있는 행만 남김
# ============================================================
df = df[df["시점"].astype(str).str.contains(r"\d{4}", regex=True)].copy()


In [12]:
# ============================================================
# [단락 6] 시점 정규화: "2000. 01" -> "2000-01"
# ============================================================
def normalize_ym(s: str):
    s = str(s).strip()
    m = re.search(r"(\d{4})\D+(\d{1,2})", s)  # 연도 + (구분자) + 월
    if not m:
        return None
    y = int(m.group(1))
    mo = int(m.group(2))
    return f"{y:04d}-{mo:02d}"

df["시점"] = df["시점"].map(normalize_ym)
df = df.dropna(subset=["시점"]).reset_index(drop=True)

print("정규화된 시점 예시:\n", df[["시점"]].head())


정규화된 시점 예시:
         시점
0  2000-01
1  2000-02
2  2000-03
3  2000-04
4  2000-05


In [14]:
# ============================================================
# [단락 7] 필요한 컬럼만 남기기 (합계 X, 3개 지표 따로 유지)
# ============================================================
df_out = df[base_cols].copy()

print("전처리 결과 상위 10개:\n", df_out.head(10))
print("전처리 결과 마지막 3개:\n", df_out.tail(3))


전처리 결과 상위 10개:
    경제활동참가율  실업률   고용률
0     61.6  6.7  57.5
1     61.1  6.5  57.2
2     60.7  5.9  57.1
3     60.7  5.3  57.5
4     61.5  4.5  58.8
5     61.2  4.2  58.6
6     62.0  4.6  59.1
7     61.1  4.6  58.3
8     61.7  4.5  58.9
9     61.4  4.2  58.8
전처리 결과 마지막 3개:
      경제활동참가율  실업률   고용률
273     62.4  3.1  60.5
274     62.6  2.8  60.8
275     62.5  2.8  60.8


In [16]:
# ============================================================
# [단락 8] 저장
# ============================================================
out_path = r"C:\ai\lecNote\1st_Project\data\고용지표_3개지표만_정리.csv"
df_out.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"✅ 저장 완료: {out_path}")

✅ 저장 완료: C:\ai\lecNote\1st_Project\data\고용지표_3개지표만_정리.csv


In [17]:
# ============================================================
# [단락 1] 라이브러리
# ============================================================
import pandas as pd


In [19]:
# ============================================================
# [단락 2] 한글 CSV 로드 함수 (인코딩 자동)
# ============================================================
def read_korean_csv(path: str) -> pd.DataFrame:
    for enc in ["utf-8-sig", "utf-8", "cp949", "euc-kr"]:
        try:
            return pd.read_csv(path, low_memory=False, encoding=enc)
        except UnicodeDecodeError:
            pass
    return pd.read_csv(path, low_memory=False, encoding="latin1")  # 최후 수단


In [21]:
# ============================================================
# [단락 3] 파일 로드
# ============================================================
food_path = r"C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점.csv"
emp_path  = r"C:\ai\lecNote\1st_Project\data\고용지표_3개지표만_정리.csv"   # ✅ 시점 없는 3개 지표 파일

food = read_korean_csv(food_path)
emp  = read_korean_csv(emp_path)

print("food:", food.shape)
print("emp :", emp.shape)
print("emp columns:", emp.columns.tolist())


food: (527872, 48)
emp : (276, 3)
emp columns: ['경제활동참가율', '실업률', '고용률']


In [22]:
# ============================================================
# [단락 4] 고용지표 전처리: (시점 없음) -> 월별 시점(YYYY-MM) 자동 생성
#  - ✅ emp가 월별로 정렬되어 있고, 첫 행이 2000-01부터 시작한다고 가정
#  - 시작월이 다르면 start_ym만 수정하면 됨
# ============================================================
base_cols = ["경제활동참가율", "실업률", "고용률"]
missing = [c for c in base_cols if c not in emp.columns]
if missing:
    raise ValueError(f"고용지표 파일에 필수 컬럼이 없습니다: {missing}")

# 숫자 변환
for c in base_cols:
    emp[c] = pd.to_numeric(emp[c], errors="coerce")

# ✅ 여기만 바꾸면 시작월 변경 가능
start_ym = "2000-02"

# 월별 시점 생성 (행 수만큼)
emp["인허가_yyyy_mm"] = pd.date_range(start=start_ym, periods=len(emp), freq="MS").strftime("%Y-%m")

# 머지용 테이블
emp_key = emp[["인허가_yyyy_mm"] + base_cols].copy()

print("emp_key head:\n", emp_key.head())
print("emp_key tail:\n", emp_key.tail())


emp_key head:
   인허가_yyyy_mm  경제활동참가율  실업률   고용률
0     2000-01     61.6  6.7  57.5
1     2000-02     61.1  6.5  57.2
2     2000-03     60.7  5.9  57.1
3     2000-04     60.7  5.3  57.5
4     2000-05     61.5  4.5  58.8
emp_key tail:
     인허가_yyyy_mm  경제활동참가율  실업률   고용률
271     2022-08     62.1  2.3  60.7
272     2022-09     62.5  2.8  60.7
273     2022-10     62.4  3.1  60.5
274     2022-11     62.6  2.8  60.8
275     2022-12     62.5  2.8  60.8


In [23]:
# ============================================================
# [단락 5] 일반음식점 전처리: 인허가일자 -> YYYY-MM
# ============================================================
food["인허가일자_dt"] = pd.to_datetime(food["인허가일자"], errors="coerce")

# 혹시 20240530 / 2024.05.30 같은 형태 보정
mask_bad = food["인허가일자_dt"].isna() & food["인허가일자"].notna()
if mask_bad.any():
    tmp = food.loc[mask_bad, "인허가일자"].astype(str).str.replace(r"[^\d]", "", regex=True)
    food.loc[mask_bad, "인허가일자_dt"] = pd.to_datetime(tmp, errors="coerce", format="%Y%m%d")

food["인허가_yyyy_mm"] = food["인허가일자_dt"].dt.strftime("%Y-%m")

print(food[["인허가일자", "인허가_yyyy_mm"]].head())


        인허가일자 인허가_yyyy_mm
0  2024-05-30     2024-05
1  2025-05-02     2025-05
2  2025-05-02     2025-05
3  2025-05-02     2025-05
4  2024-05-31     2024-05


In [24]:
# ============================================================
# [단락 6] 머지: 인허가 월(YYYY-MM) 기준으로 LEFT JOIN
# ============================================================
merged = food.merge(emp_key, on="인허가_yyyy_mm", how="left")

print("merged shape:", merged.shape)
print(merged[["인허가일자", "인허가_yyyy_mm"] + base_cols].head(10))


merged shape: (527872, 53)
        인허가일자 인허가_yyyy_mm  경제활동참가율  실업률  고용률
0  2024-05-30     2024-05      NaN  NaN  NaN
1  2025-05-02     2025-05      NaN  NaN  NaN
2  2025-05-02     2025-05      NaN  NaN  NaN
3  2025-05-02     2025-05      NaN  NaN  NaN
4  2024-05-31     2024-05      NaN  NaN  NaN
5  2024-05-31     2024-05      NaN  NaN  NaN
6  2024-05-31     2024-05      NaN  NaN  NaN
7  2024-05-31     2024-05      NaN  NaN  NaN
8  2024-05-31     2024-05      NaN  NaN  NaN
9  2024-05-31     2024-05      NaN  NaN  NaN


In [25]:
# ============================================================
# [단락 7] 최종 정리: 키 컬럼 제거 (시점 컬럼 최종에서 안 남기기)
# ============================================================
merged = merged.drop(columns=["인허가_yyyy_mm", "인허가일자_dt"], errors="ignore")


In [27]:
# ============================================================
# [단락 8] 저장
# ============================================================
out_path = r"C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_고용지표머지.csv"
merged.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"✅ 저장 완료: {out_path}")


✅ 저장 완료: C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_고용지표머지.csv


In [28]:
# '실업률' 결측치 제거
df = df[df['실업률'].notna()].copy()

In [29]:
# ============================================================
# [단락 8] 저장
# ============================================================
out_path = r"C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_고용지표머지2.csv"
merged.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"✅ 저장 완료: {out_path}")

✅ 저장 완료: C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_고용지표머지2.csv


In [30]:
# ===============================
# 📌 CSV에서 불러올 컬럼 지정
# (불필요한 컬럼 제거 → 메모리 절약)
# ===============================
use_cols = [
    '인허가일자',
    '상세영업상태명',
    '폐업일자',
    '소재지전체주소',
    '도로명전체주소',
    '업태구분명',
    '좌표정보x(epsg5174)',
    '좌표정보y(epsg5174)',
    '사업장명',
    '시설총규모',
    '경제활동참가율',
    '실업률',
    '고용률'
]


In [33]:
# ============================================================
# [단락 2] use_cols만 남기기 (없는 컬럼은 자동 제외)
# ============================================================
keep = [c for c in use_cols if c in merged.columns]   # merged 대신 너 df 변수명 쓰면 됨
missing = [c for c in use_cols if c not in merged.columns]

merged = merged[keep].copy()

print("✅ 남긴 컬럼:", merged.columns.tolist())
if missing:
    print("⚠️ 파일에 없어서 제외된 컬럼:", missing)


✅ 남긴 컬럼: ['인허가일자', '상세영업상태명', '폐업일자', '소재지전체주소', '도로명전체주소', '업태구분명', '좌표정보x(epsg5174)', '좌표정보y(epsg5174)', '사업장명', '시설총규모', '경제활동참가율', '실업률', '고용률']


In [34]:
# ============================================================
# [단락 3] 저장 (원하면)
# ============================================================
out_path = r"C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_필요컬럼만.csv"
merged.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"✅ 저장 완료: {out_path}")


✅ 저장 완료: C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_필요컬럼만.csv


In [35]:
# ============================================================
# [단락 1] 인허가일자 -> datetime 변환 + YYYY-MM 생성
# ============================================================
import pandas as pd

df = merged.copy()   # ✅ 너 df 변수명이 다르면 여기만 수정

df["인허가일자_dt"] = pd.to_datetime(df["인허가일자"], errors="coerce")

# 혹시 20240530 / 2024.05.30 같은 형태 보정(필요할 때만)
mask_bad = df["인허가일자_dt"].isna() & df["인허가일자"].notna()
if mask_bad.any():
    tmp = df.loc[mask_bad, "인허가일자"].astype(str).str.replace(r"[^\d]", "", regex=True)
    df.loc[mask_bad, "인허가일자_dt"] = pd.to_datetime(tmp, errors="coerce", format="%Y%m%d")

df["인허가_ym_dt"] = df["인허가일자_dt"].dt.to_period("M").dt.to_timestamp()


In [36]:
# ============================================================
# [단락 2] 2000-02 ~ 2022-11 (월 기준)만 필터링
# ============================================================
start_dt = pd.Timestamp("2000-02-01")
end_dt   = pd.Timestamp("2022-11-01")

df = df[(df["인허가_ym_dt"] >= start_dt) & (df["인허가_ym_dt"] <= end_dt)].copy()

print("✅ 필터 후 행 수:", len(df))
print(df[["인허가일자", "인허가_ym_dt"]].head())
print(df[["인허가일자", "인허가_ym_dt"]].tail())


✅ 필터 후 행 수: 290753
         인허가일자  인허가_ym_dt
17  2020-08-03 2020-08-01
18  2020-08-03 2020-08-01
19  2020-08-03 2020-08-01
20  2020-08-03 2020-08-01
21  2020-08-03 2020-08-01
             인허가일자  인허가_ym_dt
527377  2020-12-28 2020-12-01
527384  2003-06-05 2003-06-01
527394  2015-06-26 2015-06-01
527395  2021-11-22 2021-11-01
527674  2022-10-07 2022-10-01


In [37]:
# ============================================================
# [단락 3] 임시 컬럼 제거(원하면)
# ============================================================
df = df.drop(columns=["인허가일자_dt", "인허가_ym_dt"], errors="ignore")


In [39]:
# ============================================================
# [단락 4] 저장(원하면)
# ============================================================
out_path = r"C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_200002_202211만.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"✅ 저장 완료: {out_path}")


✅ 저장 완료: C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_200002_202211만.csv


In [40]:
# ============================================================
# [단락 1] 라이브러리 + 파일 로드
# ============================================================
import pandas as pd

in_path = "서울특별시_일반음식점_200002_202211만.csv"  # ✅ 업로드한 파일명
df = pd.read_csv(in_path, low_memory=False, encoding="utf-8-sig")

print(df.shape)
print(df[["인허가일자", "폐업일자"]].head())


(290753, 13)
        인허가일자 폐업일자
0  2020-08-03  NaN
1  2020-08-03  NaN
2  2020-08-03  NaN
3  2020-08-03  NaN
4  2020-08-03  NaN


In [41]:
# ============================================================
# [단락 2] 날짜 컬럼 datetime 변환 (문자/숫자/점표기 혼합 대응)
# ============================================================
df["인허가일자_dt"] = pd.to_datetime(df["인허가일자"], errors="coerce")
df["폐업일자_dt"]   = pd.to_datetime(df["폐업일자"], errors="coerce")

# 인허가일자 보정: 20240530 / 2024.05.30 / 공백 섞임 등
mask1 = df["인허가일자_dt"].isna() & df["인허가일자"].notna()
if mask1.any():
    tmp = df.loc[mask1, "인허가일자"].astype(str).str.replace(r"[^\d]", "", regex=True)
    df.loc[mask1, "인허가일자_dt"] = pd.to_datetime(tmp, errors="coerce", format="%Y%m%d")

# 폐업일자 보정
mask2 = df["폐업일자_dt"].isna() & df["폐업일자"].notna()
if mask2.any():
    tmp = df.loc[mask2, "폐업일자"].astype(str).str.replace(r"[^\d]", "", regex=True)
    df.loc[mask2, "폐업일자_dt"] = pd.to_datetime(tmp, errors="coerce", format="%Y%m%d")

print("인허가일자_dt 결측:", df["인허가일자_dt"].isna().sum())
print("폐업일자_dt 결측:", df["폐업일자_dt"].isna().sum())


인허가일자_dt 결측: 0
폐업일자_dt 결측: 76038


In [42]:
# ============================================================
# [단락 3] 폐업_3년이내 (0/1) 생성
#  - 폐업일자 없으면 0 (미폐업으로 간주)
#  - (폐업일자 - 인허가일자) 0~1095일이면 1
# ============================================================
days_3y = 365 * 3  # 1095일

gap_days = (df["폐업일자_dt"] - df["인허가일자_dt"]).dt.days

df["폐업_3년이내"] = (
    gap_days.notna() &        # 폐업일자 존재
    (gap_days >= 0) &         # 이상치 방지(폐업일자가 인허가보다 빠른 경우 제외)
    (gap_days <= days_3y)     # 3년 이내
).astype("int8")

print(df[["인허가일자", "폐업일자", "폐업_3년이내"]].head(10))
print("라벨 분포:\n", df["폐업_3년이내"].value_counts(dropna=False))


        인허가일자 폐업일자  폐업_3년이내
0  2020-08-03  NaN        0
1  2020-08-03  NaN        0
2  2020-08-03  NaN        0
3  2020-08-03  NaN        0
4  2020-08-03  NaN        0
5  2020-08-03  NaN        0
6  2020-08-03  NaN        0
7  2014-11-21  NaN        0
8  2000-03-02  NaN        0
9  2001-03-17  NaN        0
라벨 분포:
 0    194532
1     96221
Name: 폐업_3년이내, dtype: int64


In [44]:
# ============================================================
# [단락 4] (선택) 임시 컬럼 제거 + 저장
# ============================================================
df = df.drop(columns=["인허가일자_dt", "폐업일자_dt"], errors="ignore")

out_path = r"C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_200002_202211_폐업3년이내_01.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print("✅ 저장 완료:", out_path)


✅ 저장 완료: C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_200002_202211_폐업3년이내_01.csv


In [45]:
# ============================================================
# [단락 1] 로드 + 날짜형 변환
# ============================================================
import pandas as pd

in_path = "서울특별시_일반음식점_200002_202211_폐업3년이내_01.csv"
df = pd.read_csv(in_path, low_memory=False, encoding="utf-8-sig")

df["인허가일자_dt"] = pd.to_datetime(df["인허가일자"], errors="coerce")
df["폐업일자_dt"]   = pd.to_datetime(df["폐업일자"], errors="coerce")

# 혹시 20240530 / 2024.05.30 같은 형식 섞이면 보정(필요할 때만)
mask1 = df["인허가일자_dt"].isna() & df["인허가일자"].notna()
if mask1.any():
    tmp = df.loc[mask1, "인허가일자"].astype(str).str.replace(r"[^\d]", "", regex=True)
    df.loc[mask1, "인허가일자_dt"] = pd.to_datetime(tmp, errors="coerce", format="%Y%m%d")

mask2 = df["폐업일자_dt"].isna() & df["폐업일자"].notna()
if mask2.any():
    tmp = df.loc[mask2, "폐업일자"].astype(str).str.replace(r"[^\d]", "", regex=True)
    df.loc[mask2, "폐업일자_dt"] = pd.to_datetime(tmp, errors="coerce", format="%Y%m%d")

print(df[["인허가일자", "폐업일자", "인허가일자_dt", "폐업일자_dt"]].head())


        인허가일자 폐업일자   인허가일자_dt 폐업일자_dt
0  2020-08-03  NaN 2020-08-03     NaT
1  2020-08-03  NaN 2020-08-03     NaT
2  2020-08-03  NaN 2020-08-03     NaT
3  2020-08-03  NaN 2020-08-03     NaT
4  2020-08-03  NaN 2020-08-03     NaT


In [46]:
# ============================================================
# [단락 2] 영업일수 계산
#  - 폐업일자 없으면 오늘 날짜로 계산
# ============================================================
today = pd.Timestamp.today().normalize()

end_dt = df["폐업일자_dt"].fillna(today)
df["영업일수"] = (end_dt - df["인허가일자_dt"]).dt.days

# 이상치 방지(인허가일자 결측/미래/오류 등)
df.loc[df["영업일수"] < 0, "영업일수"] = pd.NA

print(df[["인허가일자", "폐업일자", "영업일수"]].head(10))
print("영업일수 결측:", df["영업일수"].isna().sum())


        인허가일자 폐업일자    영업일수
0  2020-08-03  NaN  1981.0
1  2020-08-03  NaN  1981.0
2  2020-08-03  NaN  1981.0
3  2020-08-03  NaN  1981.0
4  2020-08-03  NaN  1981.0
5  2020-08-03  NaN  1981.0
6  2020-08-03  NaN  1981.0
7  2014-11-21  NaN  4063.0
8  2000-03-02  NaN  9440.0
9  2001-03-17  NaN  9060.0
영업일수 결측: 0


In [47]:
# ============================================================
# [단락 3] (선택) 임시 컬럼 정리 + 저장
# ============================================================
df = df.drop(columns=["인허가일자_dt", "폐업일자_dt"], errors="ignore")

out_path = r"C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_영업일수추가.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print("✅ 저장 완료:", out_path)


✅ 저장 완료: C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_영업일수추가.csv


In [64]:
# ============================================================
# [단락 1] 로드
# ============================================================
import pandas as pd

food_path = r"C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_영업일수추가.csv"
rate_path = r"C:\ai\lecNote\1st_Project\data\korea_base_rate.csv"

df = pd.read_csv(food_path, low_memory=False, encoding="utf-8-sig")
rate = pd.read_csv(rate_path, low_memory=False, encoding="utf-8-sig")

print(df.shape, rate.shape)
print(rate.head())


(290753, 15) (53, 2)
         date  base_rate
0  2000-02-10       5.00
1  2000-10-05       5.25
2  2001-02-08       5.00
3  2001-07-05       4.75
4  2001-08-09       4.50


In [65]:
# ============================================================
# [단락 2] 날짜 컬럼 datetime 변환
#  - df: 인허가일자
#  - rate: date
# ============================================================
df["인허가일자_dt"] = pd.to_datetime(df["인허가일자"], errors="coerce")
rate["date_dt"] = pd.to_datetime(rate["date"], errors="coerce")

# 혹시 df 인허가일자가 20240530 / 2024.05.30 같은 형태면 보정(필요할 때만)
mask_bad = df["인허가일자_dt"].isna() & df["인허가일자"].notna()
if mask_bad.any():
    tmp = df.loc[mask_bad, "인허가일자"].astype(str).str.replace(r"[^\d]", "", regex=True)
    df.loc[mask_bad, "인허가일자_dt"] = pd.to_datetime(tmp, errors="coerce", format="%Y%m%d")

# 금리도 숫자 변환
rate["base_rate"] = pd.to_numeric(rate["base_rate"], errors="coerce")

# 결측 제거
df = df.dropna(subset=["인허가일자_dt"]).copy()
rate = rate.dropna(subset=["date_dt", "base_rate"]).copy()


In [66]:
# ============================================================
# [단락 3] 인허가일자 기준으로 "직전 금리" 붙이기 (merge_asof)
#  - direction="backward" : 인허가일자 당일 또는 이전의 가장 최신 금리
# ============================================================
df_sorted = df.sort_values("인허가일자_dt").reset_index(drop=True)
rate_sorted = rate.sort_values("date_dt").reset_index(drop=True)

merged = pd.merge_asof(
    df_sorted,
    rate_sorted[["date_dt", "base_rate"]],
    left_on="인허가일자_dt",
    right_on="date_dt",
    direction="backward"
)

# base_rate 컬럼이 최종적으로 붙음
print(merged[["인허가일자", "인허가일자_dt", "base_rate"]].head(10))
print("base_rate 결측 개수:", merged["base_rate"].isna().sum())


        인허가일자   인허가일자_dt  base_rate
0  2000-02-01 2000-02-01        NaN
1  2000-02-01 2000-02-01        NaN
2  2000-02-01 2000-02-01        NaN
3  2000-02-01 2000-02-01        NaN
4  2000-02-01 2000-02-01        NaN
5  2000-02-01 2000-02-01        NaN
6  2000-02-01 2000-02-01        NaN
7  2000-02-01 2000-02-01        NaN
8  2000-02-01 2000-02-01        NaN
9  2000-02-01 2000-02-01        NaN
base_rate 결측 개수: 236


In [67]:
# ============================================================
# [단락 4] 정리(원하면): 머지용 컬럼 제거
# ============================================================
merged = merged.drop(columns=["인허가일자_dt", "date_dt"], errors="ignore")


In [68]:
# ============================================================
# [단락 5] 저장
# ============================================================
out_path = r"C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_영업일수추가_기준금리머지.csv"
merged.to_csv(out_path, index=False, encoding="utf-8-sig")
print("✅ 저장 완료:", out_path)


✅ 저장 완료: C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_영업일수추가_기준금리머지.csv


In [69]:
# ============================================================
# [단락 1] 로드
# ============================================================
import pandas as pd

in_path = r"C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_영업일수추가_기준금리머지.csv"
df = pd.read_csv(in_path, low_memory=False, encoding="utf-8-sig")

print("원본 shape:", df.shape)
print("결측치 개수(컬럼별 상위):\n", df.isna().sum().sort_values(ascending=False).head(20))


원본 shape: (290753, 16)
결측치 개수(컬럼별 상위):
 도로명전체주소            86910
폐업일자               76038
좌표정보x(epsg5174)     9161
좌표정보y(epsg5174)     9161
base_rate            236
소재지전체주소              223
업태구분명                 13
시설총규모                  4
사업장명                   1
인허가일자                  0
상세영업상태명                0
경제활동참가율                0
실업률                    0
고용률                    0
폐업_3년이내                0
영업일수                   0
dtype: int64


In [70]:
# ============================================================
# [단락 2] 결측치 제거: base_rate만 기준으로 제거
# ============================================================
df_clean = df.dropna(subset=["base_rate"]).copy()

print("base_rate 결측 제거 후 shape:", df_clean.shape)
print("base_rate 결측 남았는지:", df_clean["base_rate"].isna().sum())



base_rate 결측 제거 후 shape: (290517, 16)
base_rate 결측 남았는지: 0


In [71]:
# ============================================================
# [단락 3] 저장
# ============================================================
out_path = r"C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_영업일수추가_기준금리머지.csv"
df_clean.to_csv(out_path, index=False, encoding="utf-8-sig")
print("✅ 저장 완료:", out_path)


✅ 저장 완료: C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_영업일수추가_기준금리머지.csv


In [72]:
# ============================================================
# [단락 1] 숫자 변환
# ============================================================
import pandas as pd
import numpy as np

df = df_clean.copy()   # ✅ 너 DF 이름에 맞게 수정
df["시설총규모"] = pd.to_numeric(df["시설총규모"], errors="coerce")


In [73]:
# ============================================================
# [단락 2] 구간별로 10/30/50/100/150 값으로 치환(덮어쓰기)
#  - <=10 -> 10
#  - (10,30] -> 30
#  - (30,50] -> 50
#  - (50,100] -> 100
#  - (100,150] -> 150
#  - >150 -> 150  (150으로 묶음)
# ============================================================
x = df["시설총규모"]

conds = [
    x.le(10),
    x.gt(10) & x.le(30),
    x.gt(30) & x.le(50),
    x.gt(50) & x.le(100),
    x.gt(100) & x.le(150),
    x.gt(150),
]
choices = [10, 30, 50, 100, 150, 150]

df["시설총규모"] = np.select(conds, choices, default=np.nan).astype("float")

print(df["시설총규모"].value_counts(dropna=False).sort_index())
print(df[["시설총규모"]].head(10))


10.0     10612
30.0     85283
50.0     64762
100.0    77511
150.0    52345
NaN          4
Name: 시설총규모, dtype: int64
     시설총규모
236   30.0
237   30.0
238   50.0
239   30.0
240   30.0
241  100.0
242  100.0
243   30.0
244   30.0
245   30.0


In [75]:
# ============================================================
# [단락 3] (선택) 저장
# ============================================================
out_path = r"C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_시설총규모_10_30_50_100_150치환.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print("✅ 저장 완료:", out_path)


✅ 저장 완료: C:\ai\lecNote\1st_Project\data\서울특별시_일반음식점_시설총규모_10_30_50_100_150치환.csv
